# Работа модели для матчинга резюме и вакансий портала "Работа в России":

In [1]:
#Установим библиотеку, разработанную ЦПУР
#jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8899 --NotebookApp.port_retries=0
!git clone https://github.com/ellariel/rostrud_ml.git

Cloning into 'rostrud_ml'...


После скачивания корневая папка должна быть переименована в rostrud_ml

## Предварительные действия
У этого скрипта есть несколько зависимостей, их можно поставить через pip. При запуске блока кода ниже всё поставится само.

Если в какой-то момент выполнения кода вы увидите ошибку, что вам не хватает некого питоновского пакета, вы можете дописать его название в новую строчку файла requirements.txt и снова запустить этот блок.

In [ ]:
#Установка окружения
!pip install -r rostrud_ml/requirements.txt

В файле rostrud_ml/utils/all_tables_names.yml указать путь к директории, куда будут скачиваться и парситься файлы с резюме, вакансиями (сейчас там "workdir").

В файле rostrud-ml/utils/config_to_bd_example.yml указать ваши настройки для подключения к БД и поменять название файла на config_to_bd.yml

In [61]:
import requests
from tqdm import tqdm
import pandas as pd

def _getfileslist(url):
  r = requests.get(url)
  return ['data' + i.split('">data')[0] for i in r.text.split('<a href="data')[1:]]

In [62]:
base_url = 'https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/'
file_url = ['7710538364-cv/', '7710538364-professions/', '7710538364-regions/', '7710538364-industries/', '7710538364-vacancy/']

In [83]:
s = pd.DataFrame(_getfileslist(base_url + file_url[0])).rename(columns={0: 'cv'})
s = s[s['cv'].apply(lambda x: pd.notna(x) and '.gz' in x)]
s = s[s['cv'].apply(lambda x: int(x.split('data-')[1][:4]) >= 2019)]
s['month'] = s['cv'].apply(lambda x: x.split('data-')[1][:6])
s = s.groupby('month').agg('last').reset_index()

for l in file_url[1:]:
    c = l.split('-')[1][:2]
    v = pd.DataFrame(pd.Series(_getfileslist(base_url + l)).rename(c))
    v = v[v[c].apply(lambda x: pd.notna(x) and '.gz' in x)]
    v = v[v[c].apply(lambda x: int(x.split('data-')[1][:4]) >= 2019)]
    v['month'] = v[c].apply(lambda x: x.split('data-')[1][:6])
    v = v.groupby('month').agg('last').reset_index()
    s = s.set_index('month').join(v.set_index('month')).reset_index()#.drop_duplicates('files')
s.to_csv('filtered.csv', index=False)

## Загрузка в БД необходимых данных

In [84]:
filtered = pd.read_csv('filtered.csv')
len(filtered)

56

In [85]:
#Установка модулей
from rostrud_ml.process.adding_tables_psycopg import AddingDataPsycopg
from rostrud_ml.process.renewal import Renewal

In [86]:
#Cоздание таблиц в БД
db = AddingDataPsycopg()
for table in ['curricula_vitae', 'workexp', 'vacancies', 'professions', 'industries', 'regions']:
    db.create_table(table, 'project_trudvsem') #'project_trudvsem' - название схемы в вашей БД
db.conn.close()

Вы подключены к базе: postgres


Взять ссылки на файлы с данными с ftp поратала https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/:

Из директорий
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-cv/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-vacancy/ нужно выбрать подходящие файлы (имеющие в наименовании дату) - один файл резюме и один вакансий.

Подобным образом выбрать ссылки для данных о профессиях, сферах занятости, регионах из соответствующих директорий:
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-professions/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-industries/
https://opendata.trudvsem.ru/oda2Hialoephidohyie1oR6chaem1oN0quiephooleiWei1aiD/7710538364-regions/

Для каждого типа, вставив нужную ссылку, запустить следующие ячейки:
В примере: тип "резюме" и самый первый файл резюме 2018г. Необходимо поменять ссылку на наиболее актуальный файл.

In [87]:
filtered.head(1)

,month,cv,pr,re,in,va
0,201901,data-20190131T033922-structure-20161130T143000...,data-20190131T033922-structure-20161130T143000...,data-20190131T033922-structure-20161130T143000...,data-20190131T033922-structure-20161130T143000...,data-20190131T033922-structure-20161130T143000...


In [ ]:
for _, link in tqdm(filtered.iterrows(), total=len(filtered)):
  ren = Renewal('cvs', base_url + file_url[0] + link['cv'])
  ren.download()
  #ren.parse_update()
  #ren.delete()

  #if pd.notna(link['va']):
  #  ren = Renewal('vacancies', base_url + file_url[4] + link['va'])
  #  ren.download()
    #ren.parse_update()
    #ren.delete()

  ren = Renewal('professions', base_url + file_url[1] + link['pr'])
  ren.download()
  #ren.parse_update()
  #ren.delete()

  ren = Renewal('industries', base_url + file_url[2] + link['re'])
  ren.download()
  #ren.parse_update()
  #ren.delete()

  ren = Renewal('regions', base_url + file_url[3] + link['in'])
  ren.download()
  #ren.parse_update()
  #ren.delete()

  #break

100% [....................................................................] 2245375044 / 2245375044Файлы скачены: 2022-03-29
workdir\professions\2022-03-25professions.xml.gz
100% [............................................................................] 240075 / 240075Файлы скачены: 2022-03-25
workdir\industries\2022-03-25industries.xml.gz
100% [................................................................................] 5552 / 5552Файлы скачены: 2022-03-25
workdir\regions\2022-03-25regions.xml.gz


 70%|███████████████████████████████████████████████████████                        | 39/56 [12:58<1:04:26, 227.43s/it]

100% [................................................................................] 1866 / 1866Файлы скачены: 2022-03-25
workdir\cvs\2022-04-30cvs.xml.gz
 82% [........................................................            ] 1932337152 / 2340543282